In [49]:
import os

import sys
os.chdir("/Users/zyxu/Documents/py/kris")
import numpy as np
import pandas as pd
import h5py
import hdf5plugin
import func.prepro as pp
from tqdm import tqdm
import gc

import seaborn as sns
import matplotlib.pyplot as plt

custom_colors = ["#a8e6cf","#dcedc1","#ffd3b6","#ffaaa5","#ff8b94"]
palette = sns.set_palette(sns.color_palette(custom_colors))

In [3]:
DATA_DIR = "open-problems-multimodal"
SUBMIT_DIR = "submit"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

In [3]:
submission = pd.read_csv("sample_submission.csv")
print(submission.shape)
submission.head()

(65744180, 2)


,row_id,target
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


### evaluation

In [ ]:
evaluation = pd.read_csv(f"{FP_EVALUATION_IDS}")
print(evaluation.shape)
evaluation.head()

(65744180, 3)


,row_id,cell_id,gene_id
0,0,c2150f55becb,CD86
1,1,c2150f55becb,CD274
2,2,c2150f55becb,CD270
3,3,c2150f55becb,CD155
4,4,c2150f55becb,CD112


In [ ]:
evaluation.iloc[48663*140-3: 48663*140+3,: ]

,row_id,cell_id,gene_id
6812817,6812817,ad5a949989b2,CD101
6812818,6812818,ad5a949989b2,CD88
6812819,6812819,ad5a949989b2,CD224
6812820,6812820,8d287040728a,ENSG00000204091
6812821,6812821,8d287040728a,ENSG00000198938
6812822,6812822,8d287040728a,ENSG00000168495


### protein

In [15]:
# train_cite_inputs = pp.readH5pyFile(FP_CITE_TRAIN_INPUTS)
train_cite_targets = pp.readH5pyFile(FP_CITE_TRAIN_TARGETS)
test_cite_inputs = pp.readH5pyFile(FP_CITE_TEST_INPUTS)

In [16]:
train_cite_targets.keys()

dict_keys(['axis0', 'axis1', 'block0_items', 'block0_values'])

In [27]:
proteins = train_cite_targets["axis0"].astype(str)
cellId = test_cite_inputs["axis1"].astype(str)
print("num of proteins: ",train_cite_targets["axis0"].astype(str).shape)
print("number of cells: ",test_cite_inputs["axis1"].astype(str).shape)

num of proteins:  (140,)
number of cells:  (48663,)


In [24]:
eva_protein = evaluation.iloc[:48663*140]

In [25]:
eva_protein.nunique()

row_id     6812820
cell_id      48663
gene_id        140
dtype: int64

In [37]:
# check whether eva_protein is balanced matrix
for protein in tqdm(proteins[:10]):
    num_of_cells = sum(eva_protein.gene_id == protein)
    if num_of_cells != 48663:
        print("Wrong!")
for cell in tqdm(cellId[:10]):
    num_of_proteins = sum(eva_protein.cell_id == cell)
    if num_of_proteins != 140:
        print("Wrong")

100%|██████████| 10/10 [00:09<00:00,  1.08it/s]


In [ ]:
def prot_submission(preMat, eva_protein):
    numCells = preMat.shape[0]
    assert numCells == 48663
    numProt = preMat.shape[1]
    assert numProt == 140
    
    sub = pd.DataFrame({"row_id": range(48663 * 140), "target": preMat.reshape(-1)})
    return sub

test

In [14]:
mat = train_cite_targets['block0_values'][:4,:5]

In [18]:
print(mat)
mat.reshape(-1)

[[ 1.1678035   0.62253     0.10695851  0.32498938  3.331674  ]
 [ 0.81897014  0.50600946  1.078682    6.848758    3.5248847 ]
 [-0.3567033  -0.42226133 -0.82449275  1.1374946   0.5189245 ]
 [-1.2015074   0.14911485  2.0224676   6.0215945   7.2586703 ]]


array([ 1.1678035 ,  0.62253   ,  0.10695851,  0.32498938,  3.331674  ,
        0.81897014,  0.50600946,  1.078682  ,  6.848758  ,  3.5248847 ,
       -0.3567033 , -0.42226133, -0.82449275,  1.1374946 ,  0.5189245 ,
       -1.2015074 ,  0.14911485,  2.0224676 ,  6.0215945 ,  7.2586703 ],
      dtype=float32)

In [23]:
protId = train_cite_targets['axis0'].astype(str)[:5]
protId

array(['CD86', 'CD274', 'CD270', 'CD155', 'CD112'], dtype='<U11')

In [25]:
cellId = train_cite_targets['axis1'].astype(str)[:4]
cellId

array(['45006fe3e4c8', 'd02759a80ba2', 'c016c6b0efa5', 'ba7f733a4f75'],
      dtype='<U12')

In [31]:
eva = pd.DataFrame({"row_id": range(20), "target": mat.reshape(-1)})

In [32]:
eva

,row_id,target
0,0,1.167804
1,1,0.622530
2,2,0.106959
3,3,0.324989
4,4,3.331674
5,5,0.818970
6,6,0.506009
7,7,1.078682
8,8,6.848758
9,9,3.524885


### RNA

In [42]:
eva_RNA = evaluation.iloc[48663*140:]

In [43]:
eva_RNA.head()

,row_id,cell_id,gene_id
6812820,6812820,8d287040728a,ENSG00000204091
6812821,6812821,8d287040728a,ENSG00000198938
6812822,6812822,8d287040728a,ENSG00000168495
6812823,6812823,8d287040728a,ENSG00000165527
6812824,6812824,8d287040728a,ENSG00000167414


In [44]:
# train_multi_inputs = pp.readH5pyFile(FP_MULTIOME_TRAIN_INPUTS)
train_multi_targets = pp.readH5pyFile(FP_MULTIOME_TRAIN_TARGETS)
test_multi_inputs = pp.readH5pyFile(FP_MULTIOME_TEST_INPUTS)

In [47]:
RNAs = train_multi_targets["axis0"].astype(str)
cellId_RNA = test_multi_inputs["axis1"].astype(str)
print("num of RNAs: ",RNAs.shape)
print("number of cells: ",cellId_RNA.shape)

num of RNAs:  (23418,)
number of cells:  (55935,)


In [46]:
eva_RNA.nunique()

row_id     58931360
cell_id       16780
gene_id       23418
dtype: int64

In [79]:
RNAs_in_eva = eva_RNA.gene_id.unique()

In [55]:
# check whether eva_protein is balanced matrix
for RNA in tqdm(RNAs[:3]):
    num_of_cells = sum(eva_RNA.gene_id == RNA)
    print(f"{RNA} has {num_of_cells} of cells.")


 33%|███▎      | 1/3 [00:08<00:16,  8.01s/it]

ENSG00000121410 has 2506 of cells.


100%|██████████| 3/3 [00:23<00:00,  7.68s/it]

ENSG00000268895 has 2465 of cells.
ENSG00000175899 has 2500 of cells.


In [69]:
cells_in_eva = eva_RNA.cell_id.unique()

In [70]:
for cell in tqdm(cells_in_eva[:3]):
    num_of_RNA = sum(eva_RNA.cell_id == cell)
    print(f"{cell} {num_of_RNA}!")

 33%|███▎      | 1/3 [00:07<00:14,  7.05s/it]

8d287040728a 3512!


 67%|██████▋   | 2/3 [00:14<00:07,  7.05s/it]

1a7d7864fa05 3512!


100%|██████████| 3/3 [00:20<00:00,  6.94s/it]

ec5e6fb96d6d 3512!


In [90]:
test = eva_RNA.head()
test

,row_id,cell_id,gene_id
6812820,6812820,8d287040728a,ENSG00000204091
6812821,6812821,8d287040728a,ENSG00000198938
6812822,6812822,8d287040728a,ENSG00000168495
6812823,6812823,8d287040728a,ENSG00000165527
6812824,6812824,8d287040728a,ENSG00000167414


In [104]:
cell_dict = {}
for i in range(cellId_RNA.shape[0]):
    cell_dict[cellId_RNA[i]] = i

gene_dict = {}
for i,value in enumerate(RNAs):
    gene_dict[value] = i

In [109]:
def RNA_submision(preMat, eva_RNA):
    retval = []
    for i in range(eva_RNA.shape[0]):
        rowId = cell_dict[eva_RNA.iloc[i,1]]
        colId = gene_dict[eva_RNA.iloc[i,2]]
        
        retval.append({"cell_id": eva_RNA.iloc[i,1], "gene_id": eva_RNA.iloc[i,2], "target": preMat[rowId, colId]})
    return retval

In [ ]:
retval = RNA_submision(mat, eva_RNA)

In [111]:
pd.DataFrame(retval)

,cell_id,gene_id,target
0,8d287040728a,ENSG00000204091,0.0
1,8d287040728a,ENSG00000198938,0.0
2,8d287040728a,ENSG00000168495,0.0
3,8d287040728a,ENSG00000165527,0.0
4,8d287040728a,ENSG00000167414,0.0


In [108]:
test

,row_id,cell_id,gene_id
6812820,6812820,8d287040728a,ENSG00000204091
6812821,6812821,8d287040728a,ENSG00000198938
6812822,6812822,8d287040728a,ENSG00000168495
6812823,6812823,8d287040728a,ENSG00000165527
6812824,6812824,8d287040728a,ENSG00000167414
